In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("data/Chinook_Sqlite.sqlite")

In [2]:
query_top_products = """
SELECT 
    t.Name AS Product,
    SUM(il.Quantity) AS TotalSold
FROM InvoiceLine il
JOIN Track t ON il.TrackId = t.TrackId
GROUP BY t.Name
ORDER BY TotalSold DESC
LIMIT 10;
"""
df_top_products = pd.read_sql_query(query_top_products, conn)
print("\n🔹 Top-Selling Products:\n", df_top_products)


🔹 Top-Selling Products:
                      Product  TotalSold
0                The Trooper          5
1                   Untitled          4
2    The Number Of The Beast          4
3        Sure Know Something          4
4       Hallowed Be Thy Name          4
5                   Eruption          4
6          Where Eagles Dare          3
7  Welcome Home (Sanitarium)          3
8             Sweetest Thing          3
9                  Surrender          3


In [4]:
query_revenue_region = """
SELECT 
    c.Country,
    ROUND(SUM(il.Quantity * il.UnitPrice), 2) AS Revenue
FROM InvoiceLine il
JOIN Invoice i ON il.InvoiceId = i.InvoiceId
JOIN Customer c ON i.CustomerId = c.CustomerId
GROUP BY c.Country
ORDER BY Revenue DESC;
"""
df_revenue_region = pd.read_sql_query(query_revenue_region, conn)
print("\n🔹 Revenue Per Country:\n", df_revenue_region)


🔹 Revenue Per Country:
            Country  Revenue
0              USA   523.06
1           Canada   303.96
2           France   195.10
3           Brazil   190.10
4          Germany   156.48
5   United Kingdom   112.86
6   Czech Republic    90.24
7         Portugal    77.24
8            India    75.26
9            Chile    46.62
10         Ireland    45.62
11         Hungary    45.62
12         Austria    42.62
13         Finland    41.62
14     Netherlands    40.62
15          Norway    39.62
16          Sweden    38.62
17           Spain    37.62
18          Poland    37.62
19           Italy    37.62
20         Denmark    37.62
21         Belgium    37.62
22       Australia    37.62
23       Argentina    37.62


In [5]:
query_monthly_performance = """
SELECT 
    strftime('%Y-%m', i.InvoiceDate) AS Month,
    ROUND(SUM(il.Quantity * il.UnitPrice), 2) AS Revenue
FROM InvoiceLine il
JOIN Invoice i ON il.InvoiceId = i.InvoiceId
GROUP BY Month
ORDER BY Month;
"""
df_monthly_performance = pd.read_sql_query(query_monthly_performance, conn)
print("\n🔹 Monthly Performance:\n", df_monthly_performance)


🔹 Monthly Performance:
       Month  Revenue
0   2009-01    35.64
1   2009-02    37.62
2   2009-03    37.62
3   2009-04    37.62
4   2009-05    37.62
5   2009-06    37.62
6   2009-07    37.62
7   2009-08    37.62
8   2009-09    37.62
9   2009-10    37.62
10  2009-11    37.62
11  2009-12    37.62
12  2010-01    52.62
13  2010-02    46.62
14  2010-03    44.62
15  2010-04    37.62
16  2010-05    37.62
17  2010-06    37.62
18  2010-07    37.62
19  2010-08    37.62
20  2010-09    36.63
21  2010-10    37.62
22  2010-11    37.62
23  2010-12    37.62
24  2011-01    37.62
25  2011-02    37.62
26  2011-03    37.62
27  2011-04    51.62
28  2011-05    42.62
29  2011-06    50.62
30  2011-07    37.62
31  2011-08    37.62
32  2011-09    37.62
33  2011-10    37.62
34  2011-11    23.76
35  2011-12    37.62
36  2012-01    37.62
37  2012-02    37.62
38  2012-03    37.62
39  2012-04    37.62
40  2012-05    37.62
41  2012-06    37.62
42  2012-07    39.62
43  2012-08    47.62
44  2012-09    46.71
45  2012-

In [6]:
query_products_album_artist = """
SELECT 
    t.Name AS Track,
    a.Title AS Album,
    ar.Name AS Artist,
    SUM(il.Quantity) AS TotalSold
FROM InvoiceLine il
JOIN Track t ON il.TrackId = t.TrackId
JOIN Album a ON t.AlbumId = a.AlbumId
JOIN Artist ar ON a.ArtistId = ar.ArtistId
GROUP BY t.Name, a.Title, ar.Name
ORDER BY TotalSold DESC
LIMIT 5;
"""
df_products_album_artist = pd.read_sql_query(query_products_album_artist, conn)
print("\n🔹 Top Products with Album & Artist:\n", df_products_album_artist)


🔹 Top Products with Album & Artist:
                       Track  \
0              A Cor Do Sol   
1            A Melhor Forma   
2         A Novidade (Live)   
3  Abraham, Martin And John   
4                 Aces High   

                                               Album        Artist  TotalSold  
0                                Acústico MTV [Live]  Cidade Negra          2  
1                                           Acústico         Titãs          2  
2                       Quanta Gente Veio Ver (Live)  Gilberto Gil          2  
3  Seek And Shall Find: More Of The Best (1963-1981)   Marvin Gaye          2  
4                                         Powerslave   Iron Maiden          2  


In [7]:
query_rank_products = """
SELECT 
    t.Name AS Product,
    SUM(il.Quantity) AS TotalSold,
    RANK() OVER (ORDER BY SUM(il.Quantity) DESC) AS RankBySales
FROM InvoiceLine il
JOIN Track t ON il.TrackId = t.TrackId
GROUP BY t.Name
ORDER BY RankBySales;
"""
df_rank_products = pd.read_sql_query(query_rank_products, conn)
print("\n🔹 Ranked Products by Sales (RANK):\n", df_rank_products.head(10))


🔹 Ranked Products by Sales (RANK):
                      Product  TotalSold  RankBySales
0                The Trooper          5            1
1                   Untitled          4            2
2    The Number Of The Beast          4            2
3        Sure Know Something          4            2
4       Hallowed Be Thy Name          4            2
5                   Eruption          4            2
6          Where Eagles Dare          3            7
7  Welcome Home (Sanitarium)          3            7
8             Sweetest Thing          3            7
9                  Surrender          3            7


In [8]:
query_rownum_products = """
SELECT 
    t.Name AS Product,
    SUM(il.Quantity) AS TotalSold,
    ROW_NUMBER() OVER (ORDER BY SUM(il.Quantity) DESC) AS RowNum
FROM InvoiceLine il
JOIN Track t ON il.TrackId = t.TrackId
GROUP BY t.Name
ORDER BY RowNum;
"""
df_rownum_products = pd.read_sql_query(query_rownum_products, conn)
print("\n🔹 Ranked Products by Sales (ROW_NUMBER):\n", df_rownum_products.head(10))

conn.close()


🔹 Ranked Products by Sales (ROW_NUMBER):
                      Product  TotalSold  RowNum
0                The Trooper          5       1
1                   Untitled          4       2
2    The Number Of The Beast          4       3
3        Sure Know Something          4       4
4       Hallowed Be Thy Name          4       5
5                   Eruption          4       6
6          Where Eagles Dare          3       7
7  Welcome Home (Sanitarium)          3       8
8             Sweetest Thing          3       9
9                  Surrender          3      10
